In [1]:
# Find anagrams. But use cupy. On Studio3? Perfect the workflow.

In [2]:
# standard libraries
from time import perf_counter_ns
import time

In [3]:

# external libraries
import pandas as pd
import numpy as np

# custom libraries
from _run_constants import *
from part_00_file_db_utils import *
from part_00_process_functions import *


# LOAD INPUT DATA

In [4]:
word_df, wg_df, letter_dict, char_matrix, \
    word_group_id_list, word_id_list, wchar_matrix = load_input_data(
        db_path=rc.DB_PATH, db_name=rc.DB_NAME,
        in_file_path=rc.IN_FILE_PATH)


...loading words into a dataframe...
...query execution took: 1.4 seconds...
...loading word groups into a dataframe...
...query execution took: 1.32 seconds...
...loading the letter dictionary...
...loading the char matrix...
...subsetting the char matrix...


# let's process 1000 rows using a single lookup using the full matrix

In [5]:
n_possible_anagrams = int(1e6)

In [6]:
f"{n_possible_anagrams :,}"

'1,000,000'

# Using the CPU

In [7]:
# sample 1000 rows
n_samples = 1000
sample_wg_id = wg_df['word_group_id'].sample(n = n_samples, random_state = 42).to_list()
if 746 not in sample_wg_id:
    sample_wg_id.append(746)
    #sample_wg_id = np.aray(sample_wg_id)

# establish counters for record keeping
output_list = np.full(shape=(n_possible_anagrams, 2),
                          fill_value=-1, dtype=int)

row_count = 0
anagram_pair_count = 0
intermediate_to_word_count = collections.Counter()

for wg_id in sample_wg_id:
    # identify parent words
    outcome = wchar_matrix - wchar_matrix[wg_id, ]
    
    # compute the score by finding where rows, across all columns, are GTE 0
    outcome_indices = np.all(outcome >= 0, axis=1)
    outcome = None

    n_from_words = outcome_indices.sum()

    if n_from_words >= 1:
        # extract anagrams based on index values    
        outcome_word_id_list = word_group_id_list[outcome_indices]    

        # we have matches
        # the focal word
        curr_output_list = np.zeros(shape=(n_from_words, 2), dtype=int)

        # update the output list with the word_id_list - these are from/parent words
        curr_output_list[:, 0] = outcome_word_id_list

        # update with the word_id - this is the to/child word
        curr_output_list[:, 1] = wg_id

        # enumerate the from/parent wordsds
        new_anagram_pair_count = anagram_pair_count + n_from_words

        output_list[anagram_pair_count:new_anagram_pair_count,
                    :] = curr_output_list

        # n_to_word_counter = collections.Counter(output_list[:, 0])
        intermediate_to_word_count.update(outcome_word_id_list.tolist())

        # set the anagram pair count
        anagram_pair_count = new_anagram_pair_count
    
    row_count += 1

    if row_count % 100 == 0:
        print(row_count)
print('truncating list')
output_indices = np.all(output_list >= 0, axis=1)
output_list = output_list[output_indices,]
del output_indices
output_list.shape

100
200
300
400
500
600
700
800
900
1000
truncating list


(367672, 2)

In [8]:
from_word_counter = collections.Counter(output_list[:,1])
to_word_counter = collections.Counter(output_list[:,0])

In [9]:
# the number of from word groups
from_word_counter[746]

26

# build a selector

In [10]:
# let's split the matrix, gather the values for each split, and then combine
n_subset_letters = 3
wg_df["letter_selector"] = wg_df["letter_group_ranked"].str[:n_subset_letters]

letter_selector_list = wg_df["letter_selector"].unique()
letter_selector_list.sort()
letter_selector_id_dict = {ls: i_ls for i_ls, ls in enumerate(letter_selector_list)}

wg_df["letter_selector_id"] = wg_df["letter_selector"].map(letter_selector_id_dict)
# here's the thing: I need to be able to identify on a single matrix the rows that match various conditions.
# I can't step through it and create objects at abandon. 
# so, given our wchar_matrix: what are the rows that match to such and such?
# we can add three columns to track this... 

In [11]:
wg_df.head()

,word,lcase,n_chars,first_letter,word_id,word_group_id,letter_group,letter_group_ranked,word_group_count,letter_selector,letter_selector_id
0,A,a,1,a,0,0,a,a,1,a,0
1,aa,aa,2,a,1,1,a,a,1,a,0
2,aal,aal,3,a,2,2,al,la,2,la,1081
3,aalii,aalii,5,a,3,3,ail,lai,1,lai,1083
4,aam,aam,3,a,4,4,am,ma,2,ma,1114


In [12]:
# load letter ranks
sql = 'select letter, total_letter_rank from letter_count;'

In [13]:
lr_df = query_db(sql = sql, db_path=rc.DB_PATH, db_name=rc.DB_NAME)

...query execution took: 0.0 seconds...


In [14]:
wg_df.head()

,word,lcase,n_chars,first_letter,word_id,word_group_id,letter_group,letter_group_ranked,word_group_count,letter_selector,letter_selector_id
0,A,a,1,a,0,0,a,a,1,a,0
1,aa,aa,2,a,1,1,a,a,1,a,0
2,aal,aal,3,a,2,2,al,la,2,la,1081
3,aalii,aalii,5,a,3,3,ail,lai,1,lai,1083
4,aam,aam,3,a,4,4,am,ma,2,ma,1114


In [15]:
lr_dict = {l:r for l, r in zip(lr_df['letter'], lr_df['total_letter_rank'])}

In [16]:
def get_ls_index(ls:str):
    outcome = [-1, -1, -1]
    for i_l, l in enumerate(ls):        
        outcome[i_l] = letter_dict[l]
    return outcome

In [17]:
col_names = ['letter_selector_id', 'letter_selector']

In [18]:
ls_df = wg_df[col_names].groupby(col_names).agg(np.size).to_frame(name = 'ls_count').reset_index()

In [19]:
ls_df.head()


,letter_selector_id,letter_selector,ls_count
0,0,a,4
1,1,ae,2
2,2,ai,2
3,3,b,2
4,4,ba,8


In [20]:
ls_df['ls_count'].describe()

count    2387.000000
mean      180.847926
std       424.844189
min         2.000000
25%         6.000000
50%        28.000000
75%       150.000000
max      5088.000000
Name: ls_count, dtype: float64

In [21]:
# this is effectively a column selector
ls_df['ls_index'] = ls_df['letter_selector'].map(get_ls_index)

In [22]:
ls_df.head()

,letter_selector_id,letter_selector,ls_count,ls_index
0,0,a,4,"[0, -1, -1]"
1,1,ae,2,"[0, 4, -1]"
2,2,ai,2,"[0, 8, -1]"
3,3,b,2,"[1, -1, -1]"
4,4,ba,8,"[1, 0, -1]"


In [23]:
ls_df.head()

,letter_selector_id,letter_selector,ls_count,ls_index
0,0,a,4,"[0, -1, -1]"
1,1,ae,2,"[0, 4, -1]"
2,2,ai,2,"[0, 8, -1]"
3,3,b,2,"[1, -1, -1]"
4,4,ba,8,"[1, 0, -1]"


In [24]:
ls_df['ls_count'].describe()

count    2387.000000
mean      180.847926
std       424.844189
min         2.000000
25%         6.000000
50%        28.000000
75%       150.000000
max      5088.000000
Name: ls_count, dtype: float64

In [25]:
n_possible_anagrams

1000000

In [26]:
n_possible_anagrams = load_possible_anagrams(db_path=rc.DB_PATH, db_name=rc.DB_NAME)

...query execution took: 0.0 seconds...


In [ ]:
# create the output list
output_list = np.full(shape = (n_possible_anagrams, 2), fill_value=-1)
output_time_list = []

# start counting
anagram_pair_count = 0

#for ls_id_index in range(0, 10):
for ls_row_id, ls_row in ls_df.iloc[:10].iterrows():    
    if ls_row_id % 100 == 0:
        print(ls_row_id)
    start_time = perf_counter_ns()
    
    # get letter selector id information
    ls_id = ls_row['letter_selector_id']
    ls_id_index = np.array(ls_row['ls_index'])    

    ##
    # BUILD A COLUMN SELECTOR
    ##
    
    curr_ls_id = ls_id_index[ls_id_index >= 0]
    
    ##
    # SUBSET THE wchar_matrix by column selection
    ##    
    outcome_indices = np.all(wchar_matrix[:, curr_ls_id] >= 1, axis=1)
    
    # this is the sub-matrix from which to query
    ls_wchar_matrix = wchar_matrix[outcome_indices, :]
    
    # this is the list of ids
    temp_wg_id_list = word_group_id_list[outcome_indices]
    # this is the number of records in the search space
    n_search_space = temp_wg_id_list.shape[0]
    #print(temp_wg_id_list.shape)

    def my_func(row):
        return temp_wg_id_list[np.all(a = (ls_wchar_matrix - ls_wchar_matrix[row, :]) >= 0, axis = 1)]

    #for ii in range(0, ls_wchar_matrix.shape[0]):    
    #for i_curr_wg_id, curr_wg_id in enumerate(temp_wg_id_list):
    # the current list of words featuring the set of least common letters
    curr_wg_id_list = wg_df.loc[wg_df['letter_selector_id'] == ls_id, 'word_group_id'].to_numpy()
    n_lookups = curr_wg_id_list.shape[0]
    for i_curr_wg_id, curr_wg_id in enumerate(curr_wg_id_list):
    
        #print(curr_wg_id)
        # this is indexing by row! As it should...        
        # TODO: Here
        temp_wg_id = temp_
        outcome_word_id_list = my_func(row = curr_wg_id)
                
        n_from_words = outcome_word_id_list.shape[0]
        
        if n_from_words > 0:
            outcome_word_id_list = format_output_list(outcome_word_id_list=outcome_word_id_list, wg_id=curr_wg_id)
            #print(outcome_word_id_list.shape)
            
            # enumerate the from/parent words
            new_anagram_pair_count = anagram_pair_count + n_from_words
            #print(anagram_pair_count, new_anagram_pair_count)

            output_list[anagram_pair_count:new_anagram_pair_count, :] = outcome_word_id_list

            # set the anagram pair count
            anagram_pair_count = new_anagram_pair_count

    curr_time = calc_time(time_start=start_time, round_digits=8)
    output_time_list.append([ls_id, n_search_space, n_lookups, curr_time])

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300


In [28]:
col_names =['letter_selector_id', 'n_search_space', 'n_lookups', 'curr_time']
time_df = pd.DataFrame(data = output_time_list, columns=col_names)

In [29]:
time_df.head()

,letter_selector_id,n_search_space,n_lookups,curr_time
0,0,133001,2,0.045644
1,1,84952,1,0.023877
2,2,82414,1,0.022008
3,3,35319,1,0.011744
4,4,23469,4,0.018715


In [30]:
time_df = pd.merge(left = time_df, right = ls_df)

In [31]:
time_df.head()

,letter_selector_id,n_search_space,n_lookups,curr_time,letter_selector,ls_count,ls_index
0,0,133001,2,0.045644,a,4,"[0, -1, -1]"
1,1,84952,1,0.023877,ae,2,"[0, 4, -1]"
2,2,82414,1,0.022008,ai,2,"[0, 8, -1]"
3,3,35319,1,0.011744,b,2,"[1, -1, -1]"
4,4,23469,4,0.018715,ba,8,"[1, 0, -1]"


In [32]:
time_df.tail()

,letter_selector_id,n_search_space,n_lookups,curr_time,letter_selector,ls_count,ls_index
2382,2382,397,59,0.023642,zyp,118,"[25, 24, 15]"
2383,2383,584,1,0.013702,zyr,2,"[25, 24, 17]"
2384,2384,412,12,0.012908,zys,24,"[25, 24, 18]"
2385,2385,631,9,0.010486,zyt,18,"[25, 24, 19]"
2386,2386,197,18,0.010596,zyu,36,"[25, 24, 20]"


In [33]:
time_df['curr_time'].sum()

np.float64(136.2725739)

In [34]:
time_df['n_lookups'].sum()

np.int64(215842)

In [35]:
time_df['ls_count'].sum()

np.int64(431684)

In [36]:
print('...truncating output list...')
output_indices = np.all(output_list >= 0, axis=1)
output_list = output_list[output_indices,]

...truncating output list...


In [37]:
output_list.shape

(8744217, 2)

In [38]:
from_word_counter = collections.Counter(output_list[:,1])
to_word_counter = collections.Counter(output_list[:,0])

In [39]:
# check to see if the word 'acanthology', with word_group_id 746 is in the counter
print(from_word_counter[746])
print(to_word_counter[746])

26
76
